# Preparation

In [1]:
# import all libraries
import os
import glob
import pandas as pd
import numpy as np
import math

import scipy
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

import math

import xml.etree.ElementTree as ET

#make all folders
curfolder = os.getcwd()
print(curfolder)

mtfolder = curfolder + '/Output_TimeSeries/'
print(mtfolder)

processedfolder = curfolder + '/TS_processed/'
vidfolder = curfolder + '/Output_Videos/'

# get all the files in the folder
mtfiles = glob.glob(mtfolder + '*body_world.csv')
print(mtfiles)

vidfiles = glob.glob(vidfolder + '*.mp4')
print(vidfiles)

/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/
['/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/kalt_g_ch2_g20_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/leise_g_ch2_g18_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/langsam_g_ch2_g8_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/kalt_g_ch2_g17_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/rennen_g_ch3_g17_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/rennen_g_ch3_g20_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/Donner_g_ch1

# Clean, flip, interpolate, smooth the data

In [2]:
# method to clean the data
def clean_data(file):
    sample = pd.read_csv(file)
    sample
    fileID = os.path.basename(file)
    #print(fileID)

# replace _body.csv with ''
    fileID = fileID.replace('_body_world.csv', '')
    #print(fileID)

# add fileID column to the sample
    sample['fileID'] = fileID

# get the participant ID from the fileID:
    split_string = fileID.split('ch')[1]
    split_string = split_string.split('_g')
    chain_num = split_string[0]
    split_string_gen = split_string[1].split("_compr")
    gen_num = split_string_gen[0]
    participantID = "ch" + chain_num + "g" +gen_num
    sample['participantID'] = participantID

# in the sample, get rid of all columns that contain visibility
    visibility_columns = [col for col in sample.columns if 'visibility' in col]

# get rid of all columns that contain visibility
    sample = sample.drop(visibility_columns, axis=1)
    return sample

# method to flip the data
def flip_data(df):
    cols = df.columns
    cols  = [col for col in df.columns if 'Y_' in col]
    cols = list(cols)
        # first get the vertical height of the configuration
        # we only do this for the first frame; the transformation will be applied to all frames
    maxpoint = []
    for joint in cols:
        maxpoint.append(df.loc[0, joint])
        # iterate over each joint, in each frame, to flip the y-axis
    for frame in range(len(df)):
        for joint in cols:
            ytrans = max(maxpoint) - df.loc[frame, joint] - 1
            df.loc[frame, joint] = ytrans
    return df

# interpolate the data
def interpolate(sample):
    cols = sample.columns

# put away time and fileID from cols
    cols = cols.drop(['time', 'fileID', 'participantID'])

# loop over the cols and interpolate missing data
    for col in cols:
        sample[col] = sample[col].interpolate(method='linear', x = sample['time'], limit=10)
    return cols, sample

# smooth the data
def smooth_data(sample):
    cols_upperbody = ['X_NOSE', 'Y_NOSE', 'Z_NOSE', 'X_LEFT_EYE_INNER', 'Y_LEFT_EYE_INNER', 'Z_LEFT_EYE_INNER', 'X_LEFT_EYE', 'Y_LEFT_EYE', 'Z_LEFT_EYE', 'X_LEFT_EYE_OUTER', 'Y_LEFT_EYE_OUTER', 'Z_LEFT_EYE_OUTER', 'X_RIGHT_EYE_OUTER', 'Y_RIGHT_EYE_OUTER', 'Z_RIGHT_EYE_OUTER', 'X_RIGHT_EYE', 'Y_RIGHT_EYE', 'Z_RIGHT_EYE', 'X_RIGHT_EYE_OUTER.1', 'Y_RIGHT_EYE_OUTER.1', 'Z_RIGHT_EYE_OUTER.1', 'X_LEFT_EAR', 'Y_LEFT_EAR', 'Z_LEFT_EAR', 'X_RIGHT_EAR', 'Y_RIGHT_EAR', 'Z_RIGHT_EAR', 'X_MOUTH_LEFT', 'Y_MOUTH_LEFT', 'Z_MOUTH_LEFT', 'X_MOUTH_RIGHT', 'Y_MOUTH_RIGHT', 'Z_MOUTH_RIGHT', 'X_LEFT_SHOULDER', 'Y_LEFT_SHOULDER', 'Z_LEFT_SHOULDER', 'X_RIGHT_SHOULDER', 'Y_RIGHT_SHOULDER', 'Z_RIGHT_SHOULDER', 'X_LEFT_ELBOW', 'Y_LEFT_ELBOW', 'Z_LEFT_ELBOW', 'X_RIGHT_ELBOW', 'Y_RIGHT_ELBOW', 'Z_RIGHT_ELBOW', 'X_LEFT_WRIST', 'Y_LEFT_WRIST', 'Z_LEFT_WRIST', 'X_RIGHT_WRIST', 'Y_RIGHT_WRIST', 'Z_RIGHT_WRIST', 'X_LEFT_PINKY', 'Y_LEFT_PINKY', 'Z_LEFT_PINKY', 'X_RIGHT_PINKY', 'Y_RIGHT_PINKY', 'Z_RIGHT_PINKY', 'X_LEFT_INDEX', 'Y_LEFT_INDEX', 'Z_LEFT_INDEX', 'X_RIGHT_INDEX', 'Y_RIGHT_INDEX', 'Z_RIGHT_INDEX', 'X_LEFT_THUMB', 'Y_LEFT_THUMB', 'Z_LEFT_THUMB', 'X_RIGHT_THUMB', 'Y_RIGHT_THUMB']
    cols_lowerbody = ['X_LEFT_HIP',
       'Y_LEFT_HIP', 'Z_LEFT_HIP', 'X_RIGHT_HIP', 'Y_RIGHT_HIP', 'Z_RIGHT_HIP',
       'X_LEFT_KNEE', 'Y_LEFT_KNEE', 'Z_LEFT_KNEE', 'X_RIGHT_KNEE',
       'Y_RIGHT_KNEE', 'Z_RIGHT_KNEE', 'X_LEFT_ANKLE', 'Y_LEFT_ANKLE',
       'Z_LEFT_ANKLE', 'X_RIGHT_ANKLE', 'Y_RIGHT_ANKLE', 'Z_RIGHT_ANKLE',
       'X_LEFT_HEEL', 'Y_LEFT_HEEL', 'Z_LEFT_HEEL', 'X_RIGHT_HEEL',
       'Y_RIGHT_HEEL', 'Z_RIGHT_HEEL', 'X_LEFT_FOOT_INDEX',
       'Y_LEFT_FOOT_INDEX', 'Z_LEFT_FOOT_INDEX', 'X_RIGHT_FOOT_INDEX',
       'Y_RIGHT_FOOT_INDEX', 'Z_RIGHT_FOOT_INDEX']

# smooth upperbody and face with savgol 20,4
    for col in cols_upperbody:
        sample[col] = scipy.signal.savgol_filter(sample[col], 20, 4)

# smooth lowerbod with savgol 30,3
    for col in cols_lowerbody:  
        sample[col] = scipy.signal.savgol_filter(sample[col], 10, 1)
    return(sample)


# Obtain heights and normalize data


In [3]:
#obtains the height of the body and adds in a new mid shoulder point and mid ankle point
def obtain_height_by_ankle(sample):
    #obtain left shoulder values
    x_left = sample['X_LEFT_SHOULDER'].iloc[0]
    y_left = sample['Y_LEFT_SHOULDER'].iloc[0]
    z_left = sample['Z_LEFT_SHOULDER'].iloc[0]
    left_shoulder = (x_left, y_left, z_left)
    left_shoulder = np.array(left_shoulder)

   #obtain right shoulder values
    x_right = sample['X_RIGHT_SHOULDER'].iloc[0]
    y_right = sample['Y_RIGHT_SHOULDER'].iloc[0]
    z_right = sample['Z_RIGHT_SHOULDER'].iloc[0]
    right_shoulder = (x_right, y_right, z_right)
    right_shoulder = np.array(right_shoulder)

    #calculate the mid ankle point
    new_shoulder_point = (left_shoulder + right_shoulder)/2

    # obtain left ankle values
    x_left = sample['X_LEFT_ANKLE'].iloc[0]
    y_left = sample['Y_LEFT_ANKLE'].iloc[0]
    z_left = sample['Z_LEFT_ANKLE'].iloc[0]
    left_ankle = (x_left, y_left, z_left)
    left_ankle = np.array(left_ankle)

    # obtain right ankle values
    x_right = sample['X_RIGHT_ANKLE'].iloc[0]
    y_right = sample['Y_RIGHT_ANKLE'].iloc[0]
    z_right = sample['Z_RIGHT_ANKLE'].iloc[0]
    right_ankle = (x_right, y_right, z_right)
    right_ankle = np.array(right_ankle)

    # calculate the mid ankle point 
    new_ankle_point = (left_ankle + right_ankle)/2

    sample['X_MID_ANKLE'] = new_ankle_point[0]
    sample['Y_MID_ANKLE'] = new_ankle_point[1]
    sample['Z_MID_ANKLE'] = new_ankle_point[2]

    # length of body for 3 dimensions (identical result as above)
    length_of_body3 = math.sqrt(((new_shoulder_point[0]-new_ankle_point[0])**2)+((new_shoulder_point[1]-new_ankle_point[1])**2)+((new_shoulder_point[2]-new_ankle_point[2])**2))

    # adds the points to the sample sheet
    sample['X_MID_SHOULDER'] = new_shoulder_point[0]
    sample['Y_MID_SHOULDER'] = new_shoulder_point[1]
    sample['Z_MID_SHOULDER'] = new_shoulder_point[2]

    # adds the points to the sample sheet
    sample['X_MID_ANKLE'] = new_ankle_point[0]
    sample['Y_MID_ANKLE'] = new_ankle_point[1]
    sample['Z_MID_ANKLE'] = new_ankle_point[2]

    return length_of_body3

# method to normalize by length of body
def normalize_by_body(sample, length_of_body):
    cols = sample.columns
    # put away time and fileID and participantID from cols
    cols = cols.drop(['time', 'fileID', 'participantID'])
    for col in cols: 
        sample[col] = sample[col] / length_of_body
    return(sample)



# 3D Animation

In [4]:

## plot all the keypoints in 3d space
def animation(sample):
# import pandas as pd
    import glob
    import os
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    from matplotlib.animation import FuncAnimation
    import ffmpeg

    # 3D tracking checking
    fileexample = sample
    #fileexample_hands = sample_h
    #print('Well be looking at the following file: ' + fileexample)

    MT_tracking = sample
    #MT_tracking_hands = sample_h

    # Create a figure and axis
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Define the number of frames
    num_frames = len(MT_tracking)

    # Define the scatter plot
    scatter = ax.scatter([], [], [], marker='o')
    #scatter2 = ax.scatter([], [], [], marker='o')

# Update function for animation
    def update(frame):
        ax.clear()
        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_zlabel('Z Label')
        ax.set_title('3D Animation')
        
        # Set the limits of the axes based on the maximum values
        ax.set_xlim3d(-1.1, 1.1)
        ax.set_ylim3d(-.8, .8)
        ax.set_zlim3d(-1, .5) #we need to flip the vertical
        
        # Plot the data for the current frame
        frame_data = MT_tracking.iloc[frame]
        frame_data_hands = MT_tracking.iloc[frame]
        x = frame_data.filter(like='X_')
        x_hands = frame_data_hands.filter(like='X_')
        y = frame_data.filter(like='Z_')
        y_hands = frame_data_hands.filter(like='Z_')
        z = frame_data.filter(like='Y_')
        z_hands = frame_data_hands.filter(like='Y_')
        # create scatter that has body in blue, and hands in red
        #scatter = ax.scatter(x, y, z, color='blue')
        scatter = ax.scatter(x_hands, y_hands, z_hands, color='red')

        return scatter

# Create the animation
    ani = FuncAnimation(fig, update, frames=num_frames, interval=1000/60)

# display the animation
    ani.save('3d_animation.mp4', writer='ffmpeg')

# Derive Speed and Smooth

In [5]:
# derives the speed and smooths (and changes from cm/frame/second to cm/second)
def derive_speed_smooth(cols, sample, sr):
    speedcols = [col.replace('Y_', '') for col in cols]
    speedcols = [col.replace('X_', '') for col in speedcols]
    speedcols = [col.replace('Z_', '') for col in speedcols]

# keep only unique values
    speedcols = list(set(speedcols))

    for col in speedcols:
        x = sample['X_' + col]
        y = sample['Y_' + col]
        z = sample['Z_' + col]

    # calculate speed
        sample[col + '_speed'] = np.insert(np.sqrt(np.diff(x) ** 2 + np.diff(y) ** 2 + np.diff(z) ** 2), 0, 0)
        # FLAGGED: multiply by sr to get to cm/s
        #switch to centimeters/second
        sample[col + '_speed'] = sample[col + '_speed']*sr
    #  smooth speed:
    for col in speedcols:
        sample[col + '_speed'] = scipy.signal.savgol_filter(sample[col + '_speed'], 20, 4)
    # OR
    #sample[col + '_speed'] = scipy.signal.savgol_filter(sample[col + '_speed'], 30, 3)
    return(speedcols, sample)

# Derive vertical velocity and smooth

In [6]:
# Get a list of the columns that contain Y_
def derive_vertical_vel_smooth(sample):
    verticalcols = [col for col in sample.columns if 'Y_' in col]
    verticalcols = [col for col in verticalcols if 'speed' not in col] 

# calculate the velocity
    for col in verticalcols:
        sample[col + '_velocity'] = np.insert(np.diff(sample[col]), 0, 0)  
    vel_cols = [col for col in sample.columns if "_velocity" in col]
    for col in vel_cols:
        sample[col] = scipy.signal.savgol_filter(sample[col], 20, 4)
    return verticalcols, sample


# Derive Acceleration and Smooth

In [7]:
# calculate the acceleration
def derive_accel_smooth(verticalcols, sample):
    for col in verticalcols:
        sample[col + '_acceleration'] = np.insert(np.diff(sample[col + "_velocity"]), 0, 0) 
    accel_cols = [col for col in sample.columns if "_acceleration" in col]
    for col in accel_cols:
        sample[col] = scipy.signal.savgol_filter(sample[col], 10, 3)
    return (sample)

# Derive Jerk and Smooth

In [8]:
# derive jerk from acceleration 
def derive_jerk_smooth(verticalcols, sample):
    # calcuates the jerk for each vertical column
    for col in verticalcols:
        sample[col + '_jerk'] = np.insert(np.diff(sample[col + "_acceleration"]), 0, 0)
    # gets a list of all jerk columns
    jerk_cols = [col for col in sample.columns if "_jerk" in col]
    # smooths all jerk columns
    for col in jerk_cols:
        sample[col] = scipy.signal.savgol_filter(sample[col], 20, 4)
    return (sample)

# Final code to process every file in Output_TimeSeries

Clean every file

In [9]:
# This block contains code to go through every file and clean it

import os
import glob
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#get the timeseries files
curfolder = os.getcwd()
print(curfolder)

mtfolder = curfolder + '/Output_TimeSeries/'
print(mtfolder)

cleanedfolder = curfolder + '/TS_cleaned'
os.makedirs(cleanedfolder)
processedfolder = curfolder + '/TS_processed/'
vidfolder = curfolder + '/Output_Videos/'

# get all the files in the folder
mtfiles = glob.glob(mtfolder + '*body_world.csv')
print(mtfiles)

vidfiles = glob.glob(vidfolder + '*.mp4')
print(vidfiles)

os.makedirs(processedfolder, exist_ok=True)

# for loop #1 to go through every file in the timeseries folder and clean it and save it in TS_cleaned
for file in mtfiles:
    # get the last part of the path in sample
    sample = clean_data(file)
    sample = flip_data(sample)
    cols, sample = interpolate(sample)
    sample = smooth_data(sample)
    # eventually here I can add back in the height calculus but I left it out for now. See next cell. 
    '''
    participantID = (sample['participantID'].iloc[0])
    height = obtain_height(sample)
    if participantID not in dict_of_heights:
        dict_of_heights[participantID] = [height]
    else:
        dict_of_heights[participantID].append(height)
    '''
    # collect the dictionary of heights per person
    # save it mt_process
    fileID=sample['fileID'][0]
    globals()[fileID] = sample
    globals()[fileID].to_csv(os.path.join(cleanedfolder, f'{fileID}.csv'), index=False)

# new loop over ts_process
    # normalize all cols by the height of the person
    # derive all
    # save it
'''
    sr = 1/np.mean(np.diff(sample['time']))
    speedcols, sample = derive_speed_smooth(cols, sample, sr)
    verticalcols, sample = derive_vertical_vel_smooth(sample)
    sample = derive_accel_smooth(verticalcols, sample)
    sample = derive_jerk_smooth(verticalcols, sample)
    fileID = sample['fileID'][0]
    globals()[fileID] = sample
    globals()[fileID].to_csv(os.path.join(processedfolder, f'{fileID}.csv'), index=False)
'''

/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/
['/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/kalt_g_ch2_g20_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/leise_g_ch2_g18_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/langsam_g_ch2_g8_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/kalt_g_ch2_g17_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/rennen_g_ch3_g17_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/rennen_g_ch3_g20_compr.mp4_body_world.csv', '/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/Output_TimeSeries/Donner_g_ch1

"\n    sr = 1/np.mean(np.diff(sample['time']))\n    speedcols, sample = derive_speed_smooth(cols, sample, sr)\n    verticalcols, sample = derive_vertical_vel_smooth(sample)\n    sample = derive_accel_smooth(verticalcols, sample)\n    sample = derive_jerk_smooth(verticalcols, sample)\n    fileID = sample['fileID'][0]\n    globals()[fileID] = sample\n    globals()[fileID].to_csv(os.path.join(processedfolder, f'{fileID}.csv'), index=False)\n"

Create a dictionary of heights

In [11]:
# for loop #2: code to obtain the heights from every file and save in a dictionary (eventually will go back in to first for loop)
cleanedfolder = curfolder + '/TS_cleaned'

dict_of_heights = {}
cleanedfiles = glob.glob(cleanedfolder + '/*.csv')
for file in cleanedfiles:
    sample = pd.read_csv(file)
    participantID = (sample['participantID'].iloc[0])
    height = obtain_height_by_ankle(sample)
    if participantID not in dict_of_heights:
        dict_of_heights[participantID] = [height]
    else:
        dict_of_heights[participantID].append(height)

create a dictionary of mean heights

In [12]:
# code to get the mean heights
dict_of_means = {}
dict_of_sd = {}

# get the mean height
for key in dict_of_heights: 
    values = np.array(dict_of_heights[key])
    mean = np.mean(values)
    sd = np.std(values)
    #values= np.append(values, mean)
    dict_of_means[key] = mean
    dict_of_sd[key]= sd

# print them all out: 
for key in dict_of_heights:
    print(key)
    print(dict_of_heights[key])
    print(dict_of_means[key])
    print(dict_of_sd[key])

ch3g7
[1.2381818779165867, 1.2367034965858192, 1.2311689700182362, 1.2306608137548773, 1.2183866276715467]
1.2310203571894132
0.006976359097832312
ch2g12
[1.2426141758534635, 1.2361604083109503, 1.264657133994178, 1.2389682751172708, 1.2391275695406514]
1.2443055125633027
0.010379682418210547
ch3g19
[1.2411522920379476, 1.2429214246824516, 1.241296558757481, 1.2566478907036047, 1.2341295099178868]
1.2432295352198743
0.00736218708983776
ch3g12
[1.2781776246217051, 1.272484935276709, 1.2670406929837403, 1.2647829216612176, 1.263041241883215]
1.2691054832853175
0.005539808046423878
ch1g8
[1.2227330741995444, 1.2468827976613799, 1.245499788964557, 1.215804407885716, 1.2248512027305067]
1.231154254288341
0.01264471398962366
ch1g14
[1.246318279782439, 1.2464933542247054, 1.2465167485387127, 1.2550626945648842, 1.2428684173250326]
1.2474518988871548
0.004049960997272349
ch2g6
[1.2438000601255894, 1.2216040065989489, 1.2236594701284589, 1.2364166498135654, 1.2281054555931383]
1.230717128451940

In [13]:
# check the mean of a certain one. 
dict_of_means['ch1g1']

np.float64(1.2535310635143264)

Normalize all data by mean heights and derive features. 
saves to TS_processed folder

In [14]:
# for loop #3: code to normalize all of the data based on the means and derive features. Saves to TS_processed folder. 
for file in cleanedfiles:
    print(file)
    sample = pd.read_csv(file)
    # normalize all data based on means
    participantID = (sample['participantID'].iloc[0])
    mean_body_length = dict_of_means[participantID]
    sample = normalize_by_body(sample, mean_body_length)

    # derive the kinematic features
    sr = 1/np.mean(np.diff(sample['time']))
    speedcols, sample = derive_speed_smooth(cols, sample, sr)
    verticalcols, sample = derive_vertical_vel_smooth(sample)
    sample = derive_accel_smooth(verticalcols, sample)
    sample = derive_jerk_smooth(verticalcols, sample)
    fileID = sample['fileID'][0]
    globals()[fileID] = sample
    globals()[fileID].to_csv(os.path.join(processedfolder, f'{fileID}.csv'), index=False)
    

/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_cleaned/langsam_g_ch3_g7_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_cleaned/Donner_g_ch2_g12_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_cleaned/kalt_g_ch3_g19_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_cleaned/langsam_g_ch3_g12_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_cleaned/langsam_g_ch1_g8_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_cleaned/kalt_g_ch1_g14_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_cleaned/leise_g_ch2_g6_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_cleaned/kalt_g_ch3_g20_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_cleaned/leise_g_ch1_g13_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_Iterate

# Functions to align data with ELAN Annotations

In [15]:
import xml.etree.ElementTree as ET

# Function to parse elan file
def parse_eaf_file(eaf_file, rel_tiers):
    tree = ET.parse(eaf_file)
    root = tree.getroot()

    annotations = []
    relevant_tiers = {rel_tiers}
    
    time_order = root.find('TIME_ORDER')
    if time_order == None: 
        print ("no time order")
        print()
        annotations.append({
                        'tier_id': 'na',
                        'annotation_id': 'na',
                        'start_time': 'na',
                        'end_time': 'na',
                        'annotation_value': None
                    })
    else:
        time_slots = {time_slot.attrib['TIME_SLOT_ID']: time_slot.attrib['TIME_VALUE'] for time_slot in time_order}


        for tier in root.findall('TIER'):
            tier_id = tier.attrib['TIER_ID']
            if tier_id in relevant_tiers:
                for annotation in tier.findall('ANNOTATION/ALIGNABLE_ANNOTATION'):
                    # Ensure required attributes are present
                    if 'TIME_SLOT_REF1' in annotation.attrib and 'TIME_SLOT_REF2' in annotation.attrib:
                        ts_ref1 = annotation.attrib['TIME_SLOT_REF1']
                        ts_ref2 = annotation.attrib['TIME_SLOT_REF2']
                        # Get annotation ID if it exists, otherwise set to None
                        ann_id = annotation.attrib.get('ANNOTATION_ID', None)
                        annotation_value = annotation.find('ANNOTATION_VALUE').text.strip()
                        annotations.append({
                            'tier_id': tier_id,
                            'annotation_id': ann_id,
                            'start_time': time_slots[ts_ref1],
                            'end_time': time_slots[ts_ref2],
                            'annotation_value': annotation_value
                        })

    return annotations

# Function to load annotations into csv
def fillAnno(TSfile, ANNOfile, colname):
    TSfile[colname] = None
    for row in ANNOfile.iterrows():
        start = float(row[1][0])
        end = float(row[1][1])
        value = str(row[1][2])
        TSfile.loc[(TSfile['time'] >= start) & (TSfile['time'] <= end), colname] = value

Code to Align data with ELAN

In [16]:
# CREATE TXT FILE FOR EACH TIER FROM ELAN

################
#### arms ######
################
'''
import os
import glob
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


curfolder = os.getcwd()
print(curfolder)

mtfolder = curfolder + '/Output_TimeSeries/'
print(mtfolder)

cleanedfolder = curfolder + '/TS_cleaned'
os.makedirs(cleanedfolder, exist_ok = True)
processedfolder = curfolder + '/TS_processed/'
vidfolder = curfolder + '/Output_Videos/'

# get all the files in the folder
mtfiles = glob.glob(mtfolder + '*body_world.csv')
print(mtfiles)

vidfiles = glob.glob(vidfolder + '*.mp4')
print(vidfiles)

os.makedirs(processedfolder, exist_ok=True)
'''

annofolder_manu = curfolder + '/ELAN_anno/'
annofiles_manu= glob.glob(annofolder_manu + '*.eaf')

annotations_groundTruth = curfolder + '/annotations_groundTruth/'
os.makedirs(annotations_groundTruth)
arms_anno = curfolder + '/annotations_groundTruth/arms_annotations.txt'

 # skip files that we aren't including: 
toskip = [annofolder_manu + 'rennen_g_ch1_g17_compr.eaf', annofolder_manu + 'langsam_g_ch2_g8_compr.eaf']

with open(arms_anno, 'w') as f:
    for file in annofiles_manu:
        print('working on ' + file)
        # get the filename as the last element
        filename = file.split('\\')[-1]
        # replace _ELAN_tiers.eaf with ''
        filename = filename.replace('_ELAN_tiers.eaf', '')
        print('filename:')
        print(filename)
        print(toskip[0])
        if filename in toskip:
            print("this is faulty file")
            continue
        # parse the file
        annotations = parse_eaf_file(file, 'arms')
        # write the annotations
        for annotation in annotations:
            f.write(f"{annotation['start_time']}\t{annotation['end_time']}\t{annotation['annotation_value']}\t{filename}\n")

###################
#### upper body####
###################

upperbody_anno = curfolder + '/annotations_groundTruth/upperbody_annotations.txt'

with open(upperbody_anno, 'w') as f:
    for file in annofiles_manu:
        print('working on ' + file)
        # get the filename as the last element
        filename = file.split('\\')[-1]
        # replace _ELAN_tiers.eaf with ''
        filename = filename.replace('_ELAN_tiers.eaf', '')
        # parse the file
        if filename in toskip:
            print("this is faulty file")
            continue
        annotations = parse_eaf_file(file, 'upper_body')
        # write the annotations
        for annotation in annotations:
            f.write(f"{annotation['start_time']}\t{annotation['end_time']}\t{annotation['annotation_value']}\t{filename}\n")

###################
#### lower body####
###################

lowerbody_anno = curfolder + '/annotations_groundTruth/lowerbody_annotations.txt'

with open(lowerbody_anno, 'w') as f:
    for file in annofiles_manu:
        print('working on ' + file)
        # get the filename as the last element
        filename = file.split('\\')[-1]
        # replace _ELAN_tiers.eaf with ''
        filename = filename.replace('_ELAN_tiers.eaf', '')
        if filename in toskip:
            print("this is faulty file")
            continue
        # parse the file
        annotations = parse_eaf_file(file, 'lower_body')
        # write the annotations
        for annotation in annotations:
            f.write(f"{annotation['start_time']}\t{annotation['end_time']}\t{annotation['annotation_value']}\t{filename}\n")

###################
##### head ########
###################

head_anno = curfolder + '/annotations_groundTruth/head_annotations.txt'

with open(head_anno, 'w') as f:
    for file in annofiles_manu:
        print('working on ' + file)
        # get the filename as the last element
        filename = file.split('\\')[-1]
        # replace _ELAN_tiers.eaf with ''
        filename = filename.replace('_ELAN_tiers.eaf', '')
        print(filename)
        print(toskip[0])
        if filename in toskip:
            print("this is faulty file")
            continue
        # parse the file
        annotations = parse_eaf_file(file, 'head_mov')
        # write the annotations
        for annotation in annotations:
            f.write(f"{annotation['start_time']}\t{annotation['end_time']}\t{annotation['annotation_value']}\t{filename}\n")


working on /Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_anno/Donner_g_ch3_g15_compr.eaf
filename:
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_anno/Donner_g_ch3_g15_compr.eaf
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_anno/rennen_g_ch1_g17_compr.eaf
working on /Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_anno/langsam_g_ch1_g20_compr.eaf
filename:
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_anno/langsam_g_ch1_g20_compr.eaf
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_anno/rennen_g_ch1_g17_compr.eaf
working on /Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_anno/leise_g_ch3_g11_compr.eaf
filename:
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_anno/leise_g_ch3_g11_compr.eaf
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_anno/rennen_g_ch1_g17_compr.eaf
working on /Users/gillianros

Merge the annotations with the data

In [17]:
# # ADD THE MOVEMENT ANNOTATION FROM TXTFILE TO THE DATAFRAME/CSV FILE THAT WE CREATED BY THE PROCESSING SCRIPT
# These are the annotations per tier that we just created from manual annotations
arms_anno = curfolder + '/annotations_groundTruth/arms_annotations.txt'
upperbody_anno = curfolder + '/annotations_groundTruth/upperbody_annotations.txt'
lowerbody_anno = curfolder + '/annotations_groundTruth/lowerbody_annotations.txt'
head_anno = curfolder + '/annotations_groundTruth/head_annotations.txt'

curfolder = os.getcwd()
processedfolder = curfolder + '/TS_processed/'
processedfiles = glob.glob(processedfolder + '*.csv')

movementfolder = curfolder + '/TS_movement/'
os.makedirs(movementfolder)


#videos that I need to leave out due to error so I will skip them
toskip = [processedfolder + 'rennen_g_ch1_g17_compr', processedfolder + 'langsam_g_ch2_g8_compr']

for file in processedfiles:
    print('working on ' + file)

    # TrialID
    trialid = file.split('\\')[-1].split('.')[0]

    if trialid in toskip:
        print("this is faulty file")
        continue

    trialid = trialid.replace('TS_processed', 'ELAN_anno')
    #trialid = trialid.replace('merged_', '')
    trialid = trialid + '.eaf'

    # Load the merged file
    merged = pd.read_csv(file)
    
    # Load the annotations as df
    arms = pd.read_csv(arms_anno, sep='\t', header=None)
    ub = pd.read_csv(upperbody_anno, sep='\t', header=None)
    lb = pd.read_csv(lowerbody_anno, sep='\t', header=None)
    head = pd.read_csv(head_anno, sep='\t', header=None)

    #print(arms)
    
    annos = [arms, ub, lb, head]

    # Loop over each tier and fill values into timeseries
    for anno_df in annos:
        # Get the annotations for the trialid
        anno_trial = anno_df[anno_df[3] == trialid] 
        
        if anno_trial.empty:
            print("empty")
            print('no annotations for ' + trialid)
            #add a column
            continue
        
        else:
            if anno_df.equals(arms):
                fillAnno(merged, anno_trial, 'arms')
            elif anno_df.equals(ub):
                fillAnno(merged, anno_trial, 'upper_body')
            elif anno_df.equals(lb):
                fillAnno(merged, anno_trial, 'lower_body')
            elif anno_df.equals(head):
                fillAnno(merged, anno_trial, 'head_mov')
            else:
                print('something went wrong')

    df = merged.copy()
    # check if each column exists in the dataframe
    col1 = 'arms'
    col2 = 'upper_body'
    col3 = 'lower_body'
    col4 = 'head_mov'
    # create a column if it doesn't exist. If it does, switch the labels. 
    if col1 not in df:
        df[col1] = 'no movement'
    else:
        df['arms'] = df['arms'].fillna('no movement')

    if col2 not in df:
        df[col2] = 'no movement'
    else:
        df['upper_body'] = df['upper_body'].fillna('no movement')

    if col3 not in df:
        df[col2] = 'no movement'
    else:
        df['lower_body'] = df['lower_body'].fillna('no movement')

    if col4 not in df:
        df[col4] = 'no movement'
    else:
        df['head_mov'] = df['head_mov'].fillna('no movement')

    # save the dataframe to TS movement
    fileID = trialid.replace('/ELAN_anno/', '/TS_movement/')
    print(fileID)
    globals()[fileID] = df
    globals()[fileID].to_csv(os.path.join(movementfolder, f'{fileID}.csv'), index=False)




working on /Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_processed/langsam_g_ch3_g7_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_movement/langsam_g_ch3_g7_compr.eaf
working on /Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_processed/Donner_g_ch2_g12_compr.mp4.csv
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_movement/Donner_g_ch2_g12_compr.eaf
working on /Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_processed/kalt_g_ch3_g19_compr.mp4.csv
empty
no annotations for /Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_anno/kalt_g_ch3_g19_compr.eaf
/Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_movement/kalt_g_ch3_g19_compr.eaf
working on /Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/TS_processed/langsam_g_ch3_g12_compr.mp4.csv
empty
no annotations for /Users/gillianrosenberg/Documents/GitHub/FLESH_IteratedLearning/ELAN_

# CODE THAT IS HELPFUL AS I AM DEBUGGING BUT CONFUSING TO READ THROUGH. YOU CAN IGNORE

In [ ]:
# code to obtain the mean of a single value DONT RUN THIS ONE UNESS YOU DELETE THE MEAN LATER. 
key = 'ch1g13'
values = np.array(dict_of_heights[key])
print(len(values))
mean = np.mean(values)
print("mean: " + str(mean))
sd = np.std(values)
print("sd: " + str(sd))
values= np.append(values, mean)
dict_of_heights[key] = values

print(dict_of_heights['ch1g13'])
#np.delete(dict_of_heights['ch1g13'],5)

In [ ]:
# unecessary cell that checks for the max and minimum of each
df_filtered = sample[[col for col in sample.columns if col.startswith('Z_')]]

print('z_min')
print((df_filtered.min()).min())

df_filtered = sample[[col for col in sample.columns if col.startswith('Z_')]]

print('z_max')
print((df_filtered.max()).max())

df_filtered = sample[[col for col in sample.columns if col.startswith('X_')]]

print('x_min')
print((df_filtered.min()).min())

df_filtered = sample[[col for col in sample.columns if col.startswith('X_')]]

print('x_max')
print((df_filtered.max()).max())

df_filtered = sample[[col for col in sample.columns if col.startswith('Y_')]]

print('y_min')
print((df_filtered.min()).min())

df_filtered = sample[[col for col in sample.columns if col.startswith('Y_')]]

print('y_max')
print((df_filtered.max()).max())

In [ ]:
# TEST COMPARING TWO DIFFERENT FILES
import os
import glob
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

curfolder = os.getcwd()
print(curfolder)

mtfolder = curfolder + '/Output_TimeSeries/'
print(mtfolder)

processedfolder = curfolder + '/TS_processed/'
vidfolder = curfolder + '/Output_Videos/'

# get all the files in the folder
mtfiles = glob.glob(mtfolder + '*body_world.csv')

vidfiles = glob.glob(vidfolder + '*.mp4')
print(vidfiles)

file = mtfolder + 'Donner_g_ch1_g13_compr.mp4_body_world.csv'

print(file)

sample = clean_data(file)
sample = flip_data(sample)
cols, sample = interpolate(sample)
sample = smooth_data(sample)
print(obtain_height(sample))

file = mtfolder + 'langsam_g_ch1_g13_compr.mp4_body_world.csv'

print(file)

sample = clean_data(file)
sample = flip_data(sample)
cols, sample = interpolate(sample)
sample = smooth_data(sample)
print(obtain_height(sample))

file = mtfolder + 'kalt_g_ch1_g13_compr.mp4_body_world.csv'

print(file)

sample = clean_data(file)
sample = flip_data(sample)
cols, sample = interpolate(sample)
sample = smooth_data(sample)
print(obtain_height(sample))

file = mtfolder + 'leise_g_ch1_g13_compr.mp4_body_world.csv'

print(file)

sample = clean_data(file)
sample = flip_data(sample)
cols, sample = interpolate(sample)
sample = smooth_data(sample)
print(obtain_height(sample))

file = mtfolder + 'rennen_g_ch1_g13_compr.mp4_body_world.csv'

print(file)

sample = clean_data(file)
sample = flip_data(sample)
cols, sample = interpolate(sample)
sample = smooth_data(sample)
print(obtain_height(sample))

In [ ]:
# test smoothing parameters for speed

import scipy
import matplotlib.pyplot as plt
x = 'LEFT_KNEE_speed'


# list possible window
window = [20]
# list possible orders
order = [4]

# col of interest
samplecol = [x]

# iterate over handcols and apply all possible combinations of window and order, save it to new df
# prepare new df
df_smooth = pd.DataFrame()

for col in samplecol:
    for win in window:
        for ord in order:
            df_smooth[col + '_savgol' + str(win) + '_' + str(ord)] = scipy.signal.savgol_filter(sample[col], win, ord)


# make R_Hand_x from df_sample a list
LKnee_x = sample[x].tolist()
#LKnee_y = sample[y].tolist()
#LKnee_z = sample[z].tolist()


# load these values into df_smooth as a new column
df_smooth[x] = LKnee_x
##df_smooth[y] = LKnee_y
#df_smooth[z] = LKnee_z

#plot all R_Hand_x columns in one plot
colstoplot = [col for col in df_smooth.columns if x in col]
for col in colstoplot:
    plt.plot(df_smooth[col], label=col, alpha=0.65)
plt.legend()
# make only timewindow 50 to 250
plt.xlim(100, 200)
plt.show()

In [ ]:
# test possible smoothing windows for vertical velocity 

import scipy
import matplotlib.pyplot as plt
x = 'Y_LEFT_SHOULDER_velocity'


# list possible window
window = [20]
# list possible orders
order = [4]

# col of interest
samplecol = [x]

# iterate over handcols and apply all possible combinations of window and order, save it to new df
# prepare new df
df_smooth = pd.DataFrame()

for col in samplecol:
    for win in window:
        for ord in order:
            df_smooth[col + '_savgol' + str(win) + '_' + str(ord)] = scipy.signal.savgol_filter(sample[col], win, ord)


# make R_Hand_x from df_sample a list
LKnee_x = sample[x].tolist()
#LKnee_y = sample[y].tolist()
#LKnee_z = sample[z].tolist()


# load these values into df_smooth as a new column
df_smooth[x] = LKnee_x
##df_smooth[y] = LKnee_y
#df_smooth[z] = LKnee_z

#plot all R_Hand_x columns in one plot
colstoplot = [col for col in df_smooth.columns if x in col]
for col in colstoplot:
    plt.plot(df_smooth[col], label=col, alpha=0.65)
plt.legend()
# make only timewindow 50 to 250
plt.xlim(100, 200)
plt.show()

In [ ]:
# test possible smoothing windows for acceleration

import scipy
import matplotlib.pyplot as plt
x = 'Y_LEFT_HEEL_acceleration'


# list possible window
window = [10]
# list possible orders
order = [3]

# col of interest
samplecol = [x]

# iterate over handcols and apply all possible combinations of window and order, save it to new df
# prepare new df
df_smooth = pd.DataFrame()

for col in samplecol:
    for win in window:
        for ord in order:
            df_smooth[col + '_savgol' + str(win) + '_' + str(ord)] = scipy.signal.savgol_filter(sample[col], win, ord)


# make R_Hand_x from df_sample a list
LKnee_x = sample[x].tolist()
#LKnee_y = sample[y].tolist()
#LKnee_z = sample[z].tolist()


# load these values into df_smooth as a new column
df_smooth[x] = LKnee_x
##df_smooth[y] = LKnee_y
#df_smooth[z] = LKnee_z

#plot all R_Hand_x columns in one plot
colstoplot = [col for col in df_smooth.columns if x in col]
for col in colstoplot:
    plt.plot(df_smooth[col], label=col, alpha=0.65)
plt.legend()
# make only timewindow 50 to 250
plt.xlim(100, 200)
plt.show()

In [ ]:
# test possible smoothing windows for jerk

import scipy
import matplotlib.pyplot as plt
x = 'Y_LEFT_WRIST_jerk'


# list possible window
window = [20]
# list possible orders
order = [4]

# col of interest
samplecol = [x]

# iterate over handcols and apply all possible combinations of window and order, save it to new df
# prepare new df
df_smooth = pd.DataFrame()

for col in samplecol:
    for win in window:
        for ord in order:
            df_smooth[col + '_savgol' + str(win) + '_' + str(ord)] = scipy.signal.savgol_filter(sample[col], win, ord)


# make R_Hand_x from df_sample a list
LKnee_x = sample[x].tolist()
#LKnee_y = sample[y].tolist()
#LKnee_z = sample[z].tolist()


# load these values into df_smooth as a new column
df_smooth[x] = LKnee_x
##df_smooth[y] = LKnee_y
#df_smooth[z] = LKnee_z

#plot all R_Hand_x columns in one plot
colstoplot = [col for col in df_smooth.columns if x in col]
for col in colstoplot:
    plt.plot(df_smooth[col], label=col, alpha=0.65)
plt.legend()
# make only timewindow 50 to 250
plt.xlim(100, 200)
plt.show()

In [ ]:
# visualization
plt.plot(sample['time'], sample['Z_LEFT_HIP'])
plt.show() 

In [ ]:
# plot all derivatives
plt.plot(sample['time'], sample['Y_LEFT_HEEL'])
plt.show()
plt.plot(sample['time'], sample['Y_LEFT_HEEL_velocity'])
plt.show()
plt.plot(sample['time'], sample['Y_LEFT_HEEL_acceleration'])
plt.show()
plt.plot(sample['time'], sample['Y_LEFT_HEEL_jerk'])
plt.show()